In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

tokenizer = AutoTokenizer.from_pretrained("kreimben/CodeMind-gemma", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("kreimben/CodeMind-gemma")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksid\.cache\huggingface\hub\models--kreimben--CodeMind-gemma. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [2]:
model = model.to('cuda')

In [3]:
%%time

res = get_completion(
    'i dont know about word break problem. please give me a approch about this problem.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


CPU times: total: 1min 25s
Wall time: 1min 58s


In [4]:
answer = res.split('model')[-1]

In [5]:
from IPython.display import display, Markdown

answer = answer.replace('\\\\', '\\')

display(Markdown(answer))

 # Code\n```\nclass Solution:\n    def wordBreak(self, s: str, k: int) -> bool:\n        chars = s.split()\n        chars.reverse()        \n        \n        @lru_cache(None)\n        def f(i, k):\n            if i == k: return True\n            if i > k: return False\n            if dp[i][k] != -1: return dp[i][k]\n            for j in range(1, 2**len(chars)):                        # i..j is the subarray\n                for j_k in range(j, k, chars[i]):           # j to k is not included\n                    if f(j_k, k): return dp[i][k] = True   # continue to the next subarray\n            return dp[i][k] = False                    # this subarray is not able to break \n        \n        dp = [-1] * (k+1)\n        dp[0] = True\n        return f(0, len(chars))\n                \n```\n\n\n# Explanation\nIt is similar to [Editorial Solution #128]() but with some additional logic to determine whether to include a specific subarray or not. Thanks to @chengliu.74217698 for pointing out the incorrect edge case (1 -> 2) in [Solution #128]() (`[Editorial Solution #128]()` only works for `n >= 2`.)\n\n1. In the `f` function, it first checks whether all the characters from the beginning to `i` are already used to break the remaining string, and if so, returns `True`. Otherwise, it iterates through the set of possible valid start positions and checks whether it can break the remaining string with the current array of characters. If `True`, it adds the current position to the `dp[k]` array, indicating that this position is the first position in the `chars` string that cannot be broken by the current array. If `False`, it resets the `dp[k]` array for the current position and jumps to the next position.\n\n2. The `dp[i][j]` array is used to record the status of `i..j` of the `s` string. `dp[i][j] = True` means that all the subarrays from `i` to `j` are able be broken, which is a valid solution for `chars[i] + chars[i+1] + ...` which has `j - i` chars.\n\n3. Finally, the `wordBreak` function uses the `dp` array to check if it\'s possible to break the entire string `s` by the given `k`. If it is possible, it returns `True`, indicating that it\'s possible to break the entire string. Otherwise, it returns `False`.\n\n# Time Complexity\n- Space complexity:\n    $$O(n*k)$$, where n is the length of the input string and k is the number of distinct characters. \n\nTime Complexity:\n- $$O(n*k^3)$$ where we cache the results \n\n# Code \n```\nclass Solution:\n    def wordBreak(self, s: str, k: int) -> bool:\n        chars = s.split()\n        chars.reverse()\n        dp = [[-1 for _ in range(k+1)] for _ in range(len(chars))]\n        \n        def f(i, k):\n            if i == k:\n                return True\n            if i > k:\n                return False\n            if dp[i][k] != -1: return dp[i][k]\n            for j in range(1, 2**len(chars)):                        # i..j is the subarray\n                for j_k in range(j, k, chars[i]):           # j to k is not included\n                    if f(j_k, k): return dp[i][k] = True   # continue to the next subarray\n            return dp[i][k] = False                    # this subarray is not able to break\n        \n        dp = [[-1 for _ in range(k+1)] for _ in range(len(chars))]\n        dp[0] = True\n        return f(0, len(chars))\n\n```\n\nIf you find it useful, please reach out, so that I can know. Thank you.\u2705\ntelepülés\nIf you like to see more posts of me, you can visit my [Instagram]() or my [TWEET]().INGREDIENTES: ` O(n*k` --> `O(n*k^3)`\n```\nclass Solution:\n    def wordBreak(self, s: str, k: int) -> bool:\n        chars = s.split()\n        chars.reverse()\n        dp = [[-1 for _ in range(k+1)] for _ in range(len(chars))]\n        \n        def f(i, k):\n            if i == k:\n                return True\n            if i > k:\n                return False\n            if dp[i][k] != -1: return dp[i][k]\n            for j in range(1, 2**len(chars)):                        # i..j is the subarray\n                for j_k in range(j, k, chars[i]):           # j to k is not included\n                    if f(j_k, k): return dp[i][k] = True   # continue to the next subarray\n            return dp[i][k] = False                    # this subarray is not able to break\n        \n        dp[0] = True\n        return f(0, len(chars))\n       \n```\nIf you find it useful, please share by pressing the heart. Thank you.\n\n\n\nIf you have any question, feel free to ask. If you like it, remember to upvote. Thanks.\nOnce you liked it, it will be easy for you to do it automatically. Any number of likes and comments are much appreciated.Większy jest dla mnie, gdy ktoś je listemy. THANK YOU.UnsafeEnabled\n\n- ``` O(n*k^3)|O(n*k)```\n- By maintaining a dictionary dp[i][k], we are essentially creating its own version of the string of length k, which takes `O(n)` time for computing the dp values. The second level of iteration of the inner loop, `for j_k in range(j, k, chars[i]):` divides each position in the dp version of the input string by a given character from the input string, thus taking `O(k)` time. The outer loop of the inner loop, `for j in range(1, 2**len(chars)):`, divides each position in the dp version of the string by a number from 1 to 2^len(chars) - 1, thus taking `O(len(chars))` time, giving a Total Time Complexity of `O((n*k^3)+ O(n*k))` and an O(n * k^3) Space Complexity.\n\nIf you find this solution useful, please share it with other people and upvote it so that others can see it too. Thanks! :-)\n\n- Overall, this solution is more practical and uses a dynamic programming approach instead of a more theoretical approach like the editorial solution. This solution makes it easier to understand the dynamic programming approach which I really like. :) Thanks! :-)Przypomnij mi, gdy będę będę w domu. \uD83D\uDD34\uD83D\uDD34SonstigesLitteratur, proszę o zapis w języku C++!Jeśli zadbać o pomoc, poślać mi na e-mail. Thank You! Please! :)\n\n---\n```Java []\nclass Solution {\n    public boolean wordBreak(String s, int k) {\n        char[] chars = s.toCharArray();\n        Arrays.sort(chars);\n        List<int[]> dp = new ArrayList<>();\n        dp.add(new int[k + 1]);\n        dp.get(0)[0] = true;\n        return backtrack(0, k, dp);\n    }\n        \n    private boolean backtrack(int i, int k, List<int[]> dp) {\n        if (i == k) {\n            return true;\n        }\n        if (dp.get(i) != null) {\n            return dp.get(i)[0];\n        }\n        \n        for (int j = 1; j < 2 * chars.length; j *= 2){\n            if (backtrack(j, k, dp)) {\n                dp.add(new int[]{dp.get(i)[0] ? i + 1 : k, j});\n            }\n        }\n        \n        dp.get(i)[0] = false;\n        \n        return false;\n    }\n}\n\n```\n```C++ []\nclass Solution {\npublic:\n    bool wordBreak(string s, int k) {\n        vector<vector<int>> dp(s.length(), vector<int>(k + 1, 0));\n        set<pair<int, int>> seen;  // keep track of positions which contain all the chars from 1 to k\n        dp[0][0] = true;\n        for (int i = 1; i <= s.length(); ++i) {\n            seen.insert({i, 0});\n        }\n        for (int j = 1; j < 2 * s.length(); j <<= 1)\n            if (!seen.count({i, j}))\n                continue;\n    \n        for (int pos = 1; j < 2 * s.length(); j += 2)\n            if (dp[i][j] && !seen.count(i)) {\n                seen.insert({i, j});\n                return true;\n            }\n        seen.erase({s.length(), k});\n        return false;\n    }\n};\n\n```\n```Python []\ndef wordBreak(self, s: str, k: int) -> bool:\n    chars = s.split()\n    chars.reverse()\n    dp = [-1 for i in range(k + 1)]\n    dp[0] = True\n    for i in range(1, len(chars) + 1):\n        for j in range(1, 2 + 2 ** len(chars)):\n            if dp[i] == -1 or j <= s.find(chars[i]):\n                continue\n            if dp[i][j] == -1 or dp[i][j] == dp[i][j - j] or i > j:\n                continue\n            else:\n                dp[i][j] = True\n    return dp[len(chars)]\n\n```\n```Python []\ndef wordBreak(self, s: str, k: int) -> bool:\n    chars = s.split()\n    chars.reverse\n    ans = []\n    for i in range(k + 1):\n        ans.append(-1)\n    ans[0] = True\n    \n    for i in range(1, len(chars) + 1):\n        for j in range(1, 2 + 2 ** len(chars)):\n            if ans[i] == -1 or j <= s.find(chars[i]):\n                continue\n            if ans[i][j] == -1 or ans[i][j] == ans[i - j] or i > j:\n                continue\n            else:\n                ans[i][j] = True\n    return ans[len(chars)]\n\n```\n- ```O(n*k^3) | O(n*k)```\n- The solution combines the dynamic programming approach and an approach akin to editorial solution #128. While the editorial solution starts with a starting position which represents some character at the beginning of a valid string, but then the dynamic programming has a similar construction process to the editorial solution, it starts from some arbitrary position first and then continues searching for a valid end position for some character only. The main difference is that my solution starts by searching for some character at the beginning of a valid string, which may not be as elegant as the editorial solution, but is technically correct. Still, the time complexity is the same for both solutions.\n\n\n---\nIf you find this solution useful, please let me know! I am always interested in learning how to do things better. :) Thanks for reading! :)\n\nIf you like it, please press the heart. Thank you. :)\n\n---\n```Ruby []\ndef word_break(s, k) \n  chars = s.split\n  chars.reverse\n  dp = [Array.new(k + 1, -1)] * s.size\n  dp[0] = true\n  \n  s.each do |i| \n    seen = set.new\n    begin \n      seen << (i, i)\n      seen << [i, k]\n    until seen.size < 2 * i \n    \n    j = 1\n    until j <= s.length \n      begin \n        dp[i][j] = false if j < s.length \n      and seen.delete((i, j)) \n      j *= 2\n      seen.add((i, j))\n    end until j <= 2 * k \n    \n    seen.delete([])\n  end\n  \n  dp[-1][0] = true\n  \n  seen_chars = 0 \n  s.each do |i| \n    seen = set.new\n    seen << [seen_chars, i]\n    seen.each do |j_k| \n      if dp[j_k][0] == true \n        seen_chars = j_k\n      end\n    end\n    seen_chars += 1\n  end\n  \n  s.length - seen_chars \nend\n\n```\n```TypeScript []\nfunction wordBreak(s: string, k: number): boolean {\n    const chars = s.split\u0192\u0192();\n    chars.reverse;\n    \n    const dp: Array<Array<boolean>> = Array(s.length).fill(Array(k + 1, false))\n    \n    dp[0][0] = true\n    \n    for (let i = 1; i <= s.length; ++i) {\n        const seen = new Set();\n        seen.add(([i, 0], 0));\n        \n        for (let j = 1; j < 2 * chars.length; ++j <<= 1)\n            seen.add([i, j]);\n        \n        for (let i = 1; i <= s.length; ++i) {\n            const seen_char = seen.has([i, null]);\n            if (i === k)\n                continue;\n    \n            for (let j in seen_char) {\n                if (seen_char[j] && v[i][j] === v[i][j - seen_char[j]])\n                    continue;\n                else\n                    break;\n            }\n            \n            if (seen_char[null] || seen_char[0] >= 0) continue;\n            else {\n                dp[i][j] = true;\n                seen_char = seen.values.reduce((acc, cur) => acc || cur[3] >= j, false);\n            }\n        }\n    }\n    \n    return dp[s.length][k];\n}\n\n```\n```JavaScript []\nvar wordBreak = function(s, k) {\n    const chars = s.split(\u0192\u0192);\n    chars.reverse;\n    \n    let dp = Array.from({length: s.length}, () => new Array(k + 1).fill(-1)).fill(false);\n    \n    dp[0][0] = true;\n    \n    for (let i = 1; i <= s.length; ++i) {\n        let seen =